***
# Screener

Operation:
1. Define function ```buysig()``` which scrapes stock data, calculates buy signals and returns pandas series holding information when the last buy signal happend and distance from today
2. Filter results to show only buy signals from last two days
3. Saves results into excel sheet

In [1]:
# imports
from scraper import stock_daily
from IPython.display import display
import datetime as dt
import time
import numpy as np
import pandas as pd
import indicators as ind

***
## Define ```buysig()```

* Input: empty pandas series with predefined columns
* Output: said series with filled results

Scrapes data, calculate indicators and buy signals (depending on strategy), saves results into series and outputs it

In [2]:
def buysig(result):
    # SCRAPING
    try:
        stock = stock_daily(result["ticker"], save=False)
    except:
        print(result["ticker"] + ": Exception occured during data scraping, skipped.")
        return result
    
    # CREATING BUY SIGNALS
    # Strategy SVF
    # calculating indicators
    vfi = ind.vfi(stock.data, period=30, coef=0.2, vcoef=1.5)
    fs = ind.stoch(stock.data, period=5, sk=2, sd=3)
    ss = ind.stoch(stock.data, period=21, sk=2, sd=5)
    # calculating VFI histogram trend
    window = 2
    vfi_hist = vfi["histogram"].rolling(window=window).apply(lambda x: np.polyfit(np.arange(window), x, 1)[0], raw=True).values
    with np.errstate(invalid='ignore'):
        vfi_hist = vfi_hist > 0
        # VFI trend
        vfi_trend = vfi["vfi"] > vfi["vfi_smooth"]
    vfi_conf = np.logical_or(vfi_hist, vfi_trend)
    # calculating fast stochastic trend
    window = 4
    fs_conf = fs["k"].rolling(window=window).apply(lambda x: np.polyfit(np.arange(window), x, 1)[0], raw=True).values
    with np.errstate(invalid='ignore'):
        fs_conf = fs_conf > 0
    # buy signals
    conditions = np.logical_and((ss["k"] > ss["d"]).to_numpy(), (ss["d"] >= 0).to_numpy())
    bss = np.concatenate((np.array([0]), (conditions[:-1] < conditions[1:]))).astype("int")
    # finalize buy signals
    svf = np.logical_and(np.logical_and(fs_conf, vfi_conf), bss)
    if svf.sum() == 0:
        print(result["ticker"] + ": No buying signals generated, skipped.")
        return result
    
    # Strategy S&F Oscillator
    ema3 = ind.ema(stock.data, 3, price="Typical")["EMA"].to_numpy()
    ema10 = ind.ema(stock.data, 10, price="Open")["EMA"].to_numpy()
    ema26 = ind.ema(stock.data, 26, price="Close")["EMA"].to_numpy()
    # buy signal
    bcurve = ema3-ema10
    ccurve = ema3-ema26
    lim = 0.1*stock.data["Close"].to_numpy()
    with np.errstate(invalid='ignore'):
        bmask = bcurve > 0
        cmask = (ccurve > -lim).astype("int")
    bsig = np.concatenate((np.array([0]), (bmask[:-1] < bmask[1:]))).astype("int")
    bsig = np.logical_and(bsig, cmask).astype("int")
    # VFI mask
    vfimask = (vfi["vfi"] > 0).to_numpy()
    vfimask = np.logical_or(vfimask, (vfi["histogram"] > 0).to_numpy())
    saf = np.logical_and(bsig, vfimask).astype("int")

    # SAVING DATA
    # date of last buy signal and distance from today
    # SVF
    try:
        lastindex = np.squeeze(np.where(svf == True))[-1]
        result["buy_svf"] = stock.data.loc[lastindex,"Date"]
        result["dist_svf"] = stock.data.index[-1] - lastindex
    except:
        print(result["ticker"] + ": No SVF signals generated")
    # S&F
    try:
        lastindex = np.squeeze(np.where(saf == True))[-1]
        result["buy_saf"] = stock.data.loc[lastindex,"Date"]
        result["dist_saf"] = stock.data.index[-1] - lastindex
    except:
        print(result["ticker"] + ": No S&F signals generated")

    return result

***
## Run the analysis

Tickers are stored in ```tickers.xlsx``` excel file where multiple sheets are defined. By modifying the *sheets* variable, one can choose which stocks to scan through

Sheets in the excel:
1. spy - holds all stock tickers present in the SPY ETF
2. iwm - holds top 600 holdings in the IWM ETF which follows Russell 2000
3. watchlist - list of stocks I'm personally interested in and found somewhere else, on fintwit, reddit, my own research
4. longs - stocks I have high conviction in and want to hold long term


In [3]:
# which sheets to use:
# sheets = ["spy", "iwm", "watchlist", "longs", "midcap", "largecap"] # specific scan
sheets = ["filtered"] # full nasdaq, filtered by volume and volatility
# sheets = ["test"]

# read tickers from excel file
excel = pd.read_excel("tickers.xlsx", sheet_name=sheets)

In [4]:
# define empty dataframe
data = pd.DataFrame(columns=["ticker", "type", "buy_svf", "dist_svf", "buy_saf", "dist_saf"])
# run analysis
for i in sheets:
    for j in excel[i]["ticker"].to_list():
        # define series
        ser = {"ticker": j, "type": i, "buy_svf": np.nan, "dist_svf": np.nan, "buy_saf": np.nan, "dist_saf": np.nan}
        ser = pd.Series(data=ser)
        # delay for better scrapping
        time.sleep(1)
        # calculation
        res = buysig(ser).to_frame().T
        data = data.append(res, ignore_index=True)

ABOS: No buying signals generated, skipped.
ABSI: No buying signals generated, skipped.
ADGI: No buying signals generated, skipped.
AGRI: No buying signals generated, skipped.
ALZN: No buying signals generated, skipped.
ATAI: No buying signals generated, skipped.
AVAH: Something broke during saving, skipped.
BASE: No buying signals generated, skipped.
BHG: No buying signals generated, skipped.
BLND: No buying signals generated, skipped.
BON: No buying signals generated, skipped.
BZ: Something broke during saving, skipped.
CADL: No buying signals generated, skipped.
CFLT: No buying signals generated, skipped.
CNM: No buying signals generated, skipped.
CNTA: No buying signals generated, skipped.
CNVY: No buying signals generated, skipped.
COOK: No buying signals generated, skipped.
CPOP: No buying signals generated, skipped.
CRBU: No buying signals generated, skipped.
CURV: No buying signals generated, skipped.
CVRX: No buying signals generated, skipped.
CXM: No buying signals generated,

***
## Print & Save results

In [5]:
# display results
display(data[data["distance"] < 1].drop_duplicates(subset="ticker").reset_index(drop=True))
# save data
data[data["distance"] <= 1].drop_duplicates(subset="ticker").reset_index(drop=True).to_excel("Screener-"+str(dt.date.today())+".xlsx")

,ticker,type,buy_date,distance
0,ABNB,filtered,2021-09-22,0
1,ACHV,filtered,2021-09-22,0
2,ACI,filtered,2021-09-22,0
3,ADAP,filtered,2021-09-22,0
4,ADTN,filtered,2021-09-22,0
...,...,...,...,...
208,ZKIN,filtered,2021-09-22,0
209,ZYNE,filtered,2021-09-22,0
210,LMT,filtered,2021-09-22,0
211,OEG,filtered,2021-09-22,0
